## Shor's algorithm using UBQC

In [116]:
# Load the libraries

import matplotlib.pyplot as plt
import math
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, QuantumRegister, execute, ClassicalRegister
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Shor
from qiskit.visualization import plot_histogram
from numpy.random import randint
import pandas as pd
from fractions import Fraction
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit.compiler.assembler import assemble

In this notebook we try to apply using the UBQC compiler for executing Shor's prime factorization algorithm. For this, the $\textit{Qiskit}$ implementation for Shor's algorithm is used.

### Example:

In [121]:
# Circuit for computing the prime factors of 9

circuit = shor.construct_circuit(9)
circuit.draw()

┌───┐┌─────────────┐┌───────┐
  up_0: ┤ H ├┤0            ├┤0      ├
        ├───┤│             ││       │
  up_1: ┤ H ├┤1            ├┤1      ├
        ├───┤│             ││       │
  up_2: ┤ H ├┤2            ├┤2      ├
        ├───┤│             ││       │
  up_3: ┤ H ├┤3            ├┤3      ├
        ├───┤│             ││  IQFT │
  up_4: ┤ H ├┤4            ├┤4      ├
        ├───┤│             ││       │
  up_5: ┤ H ├┤5            ├┤5      ├
        ├───┤│             ││       │
  up_6: ┤ H ├┤6            ├┤6      ├
        ├───┤│             ││       │
  up_7: ┤ H ├┤7            ├┤7      ├
        ├───┤│             │└───────┘
down_0: ┤ X ├┤8            ├─────────
        └───┘│   2^x mod 9 │         
down_1: ─────┤9            ├─────────
             │             │         
down_2: ─────┤10           ├─────────
             │             │         
down_3: ─────┤11           ├─────────
             │             │         
 aux_0: ─────┤12           ├─────────
             │             │         
 aux_1: ─────┤13           ├─────────
             │             │         
 aux_2: ─────┤14           ├─────────
             │             │         
 aux_3: ─────┤15           ├─────────
             │             │         
 aux_4: ─────┤16           ├─────────
             │             │         
 aux_5: ─────┤17           ├─────────
             └─────────────┘

Since the circuit implementation of Shor's algorithm demands some classical postprocessing, we provide a function taking a number as an input, and giving out the factors as calculated with Shor's algorithm:

In [118]:
def factorize_number(target_number):
    # Construct the circuit for Shor's algorithm with the target number
    circuit = shor.construct_circuit(target_number,5)
    
    # Measure the qubits you want to obtain the measurement results
    measurement_qubits = circuit.qubits[:int(np.log2(target_number))]  # Measure the first log2(N) qubits
    
    # Create classical registers for measurement
    creg = ClassicalRegister(len(measurement_qubits), name='c')
    
    # Add classical registers to the circuit
    circuit.add_register(creg)
    
    # Apply the Quantum Fourier Transform (QFT)
    circuit.append(QFT(len(measurement_qubits)), measurement_qubits)
    
    # Add measurement gates to the circuit
    circuit.measure(measurement_qubits, creg)
    qobj = assemble(circuit,shots=2000,memory=True)
    
    # Use a classical simulator to execute the circuit
    simulator = Aer.get_backend('aer_simulator')
    shots = 1000  # Number of times to run the circuit
    job = execute(circuit, simulator, shots=shots)
    result = job.result()
    
    # Get the measurement counts
    counts = result.get_counts(circuit)
    
    # Perform post-processing to extract the factorization result
    factors = []
    for measurement_result in counts.keys():
        binary_measurement = measurement_result[::-1]  # Reverse the binary string
        decimal_measurement = int(binary_measurement, 2)  # Convert binary to decimal
        factor = np.gcd(decimal_measurement, target_number)  # Calculate the greatest common divisor with the target number
        if factor > 1:
            factors.append(factor)
    
    # Print the factors
    print("Factors:", factors)
    
factorize_number(9)

Factors: [3, 9, 3]


### Issues with the simulation

Since the UBQC protocol works in the MBQC formalism, every circuit needs to get transformed into the corresponding measurement instructions. The current compiler provides conversion instructions for a given set of gates, while we don't have the possibility to directly convert the inverse Fourier transform or the period finding subcircuit into measurement instructions. As a solution for this, qiskit's subroutine transpile can be used, converting a circuit into an equivalent circuit using only a given set of gates, for which measurement conversion instructions are abundant. As a basis we choose rot_z, rot_x, and cx gates.


In [119]:
N = 9

backend = Aer.get_backend('aer_simulator')
quantum_instance = QuantumInstance(backend, shots=1000)
shor = Shor(quantum_instance=quantum_instance)
shorcircuit = shor.construct_circuit(N,5)
measurement_qubits = shorcircuit.qubits[:int(np.log2(N))]
creg = ClassicalRegister(len(measurement_qubits), name='c')
shorcircuit.add_register(creg)
shorcircuit.append(QFT(len(measurement_qubits)), measurement_qubits)
shorcircuit.measure(measurement_qubits,creg)
shorcircuit = transpile(shorcircuit, basis_gates = ['rz', 'rx', 'cx'])
qobj = assemble(shorcircuit,shots=1000, memory=True)

#### Conclusion
Due to the fact that the iQFT and the period finding subroutines are consisting out of a high number of gates in the given basis [RZ, RX, CX], the circuit becomes too complex for the simulation using UBQC. To overcome this problem, measurement conversion instruction for different gates could be implemented in measurement.py, or simplifications on the circuit could be performed.